In [48]:
# Import the dependencies
from sqlalchemy import create_engine, text, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
# Create a base class for declarating class definitions to produce Table objects
Base = declarative_base()

/var/folders/x7/v571_3t12ddffkm6dh_29_tr0000gn/T/ipykernel_7548/3329960063.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [49]:
# Define our pet table
class Pet(Base):
    __tablename__ = 'pet'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    age = Column(Integer)

In [50]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

FacadeDict({'pet': Table('pet', MetaData(), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(), table=<pet>), Column('type', String(), table=<pet>), Column('age', Integer(), table=<pet>), schema=None)})

In [51]:
# Create our database engine
engine = create_engine('sqlite:///pets.sqlite')

In [52]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [53]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)


## Create Data

In [54]:
# Note that adding to the session does not update the table. It queues up those queries.
session.add(Pet(name='Justin Timbersnake', type='snek', age=2))
session.add(Pet(name='Pawtrick Stewart', type='good boy', age=10))
session.add(Pet(name='Godzilla', type='iguana', age=1))
session.add(Pet(name='Marshmallow', type='polar bear', age=4))

In [55]:
# The data hasn't been added yet
engine.connect().execute(text('select * from pet')).fetchall()

[]

In [56]:
# We can use the new attribute to see the queue of data ready to go into the database
session.new

IdentitySet([<__main__.Pet object at 0x107ce1cc0>, <__main__.Pet object at 0x107c03850>, <__main__.Pet object at 0x107ce13f0>, <__main__.Pet object at 0x107c01de0>])

In [57]:
# Commit() flushes whatever remaining changes remain to the database, and commits the transaction.
session.commit()

In [58]:
# Query the database
session.query(Pet.name, Pet.type, Pet.age).all()

[('Justin Timbersnake', 'snek', 2),
 ('Pawtrick Stewart', 'good boy', 10),
 ('Godzilla', 'iguana', 1),
 ('Marshmallow', 'polar bear', 4)]

## Update Data

In [59]:
# Create a query and then run update on it
pet = session.query(Pet).filter_by(name="Marshmallow").first()
pet.age += 1

In [60]:
# For modifications, we can use the dirty attribute
session.dirty

IdentitySet([<__main__.Pet object at 0x107c01de0>])

In [61]:
# Commit Transaction
session.commit()

In [62]:
# Session is up-to-date
session.dirty

IdentitySet([])

In [63]:
# Query the database.
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Marshmallow', 'polar bear', 5)]

## Delete Data

In [64]:
# Create a query and then delete the row collected
pet = session.query(Pet).filter_by(id=4).one()
session.delete(pet)

In [65]:
session.deleted

IdentitySet([<__main__.Pet object at 0x107c01de0>])

In [66]:
session.commit()

In [67]:
session.deleted

IdentitySet([])

In [68]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1)]

In [69]:
# Uncomment to drop all tables of the database and reset the exercise
Base.metadata.drop_all(engine)

In [70]:
# Close the session
session.close()